In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch" # alternatively you can use jax or tensorflow as the backend (see installation instructions in keras documentation)
import keras

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split # helpful for splitting data into training and testing sets
from sklearn.preprocessing import StandardScaler, MinMaxScaler # helpful for scaling data to be between 0 and 1 or have a mean of 0 and standard deviation of 1


In [2]:
file_path = '/Users/ctychen/Desktop/N3AS/spectrum_data/'
#reloading data
parameters = np.concatenate([np.load(file_path + "atm_retr_params_12800_"+str(i)+".npy") for i in range(12)])
spectra_set = np.concatenate([np.load(file_path + "atm_retr_data_12800_"+str(i)+".npy") for i in range(12)])

#take the log of the spectra - seems like this helped a lot with feature identification
spectra_set = np.log(spectra_set)

parameters = parameters[~np.isnan(spectra_set).any(axis=1)]
spectra_set = spectra_set[~np.isnan(spectra_set).any(axis=1)]

print(parameters.shape)
print(spectra_set.shape)

n_samples = parameters.shape[0]
n_features = 947

max_log_flux = np.max(spectra_set, axis=1)
min_log_flux = np.min(spectra_set, axis=1)
spectra_set = (spectra_set - min_log_flux[:, np.newaxis])/(max_log_flux - min_log_flux)[:, np.newaxis]

(153590, 16)
(153590, 947)


## Moving average on the spectra

In [ ]:
def moving_average(x, n=3):
    return np.convolve(x, np.ones(n), 'valid') / n

In [ ]:
averaged_spectra_set = np.array([moving_average(spectra_set[i], n=3) for i in range(n_samples)])

## Comparing () - Abs- Diff 
Trying and comparing max-abs-diff, mean-abs-diff, median-abs-diff... and let's see how they do at finding spectra with weirdness

In [ ]:
#max-abs-diff, for the original spectra set
spectra_max_absolute_differences = np.max(np.abs(np.diff(spectra_set, axis=1)), axis=1)    

